In [9]:
from matplotlib import pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import time

torch.set_printoptions(edgeitems=2)
# 默认当前数据处理
torch.manual_seed(123)


In [10]:
# 加载数据集并进行预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                            download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                           download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
# 定义模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 32*32*3)  # 展开输入特征图
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

start_time = time.time()
# 训练模型
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
end_time = time.time()
total_time = end_time - start_time
print("运行时间为：", total_time, "秒")
print('Training finished')

[1,   100] loss: 2.271
[1,   200] loss: 2.190
[1,   300] loss: 2.094
[1,   400] loss: 2.038
[1,   500] loss: 1.974
[2,   100] loss: 1.923
[2,   200] loss: 1.886
[2,   300] loss: 1.849
[2,   400] loss: 1.825
[2,   500] loss: 1.796
[3,   100] loss: 1.759
[3,   200] loss: 1.740
[3,   300] loss: 1.739
[3,   400] loss: 1.716
[3,   500] loss: 1.702
[4,   100] loss: 1.679
[4,   200] loss: 1.680
[4,   300] loss: 1.653
[4,   400] loss: 1.629
[4,   500] loss: 1.624
[5,   100] loss: 1.607
[5,   200] loss: 1.616
[5,   300] loss: 1.592
[5,   400] loss: 1.597
[5,   500] loss: 1.572
[6,   100] loss: 1.550
[6,   200] loss: 1.565
[6,   300] loss: 1.551
[6,   400] loss: 1.549
[6,   500] loss: 1.536
[7,   100] loss: 1.518
[7,   200] loss: 1.516
[7,   300] loss: 1.504
[7,   400] loss: 1.500
[7,   500] loss: 1.508
[8,   100] loss: 1.487
[8,   200] loss: 1.477
[8,   300] loss: 1.478
[8,   400] loss: 1.451
[8,   500] loss: 1.457
[9,   100] loss: 1.446
[9,   200] loss: 1.425
[9,   300] loss: 1.443
[9,   400] 

In [12]:
# 在测试集上评估模型性能
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on test set: %d %%' % (100 * correct / total))

Accuracy on test set: 50 %
